## Creating the Index

In [2]:
print("teste")

teste


In [3]:
!pip install llama_index datasets llama-index-llms-huggingface llama-index-retrievers-bm25


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import datasets
from llama_index.core.schema import Document

guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

docs = [
    Document(
        text="\n".join([
          f"Name: {guest_dataset['name'][i]}",
          f"Relation: {guest_dataset['relation'][i]}",
          f"Description: {guest_dataset['description'][i]}",
          f"Email: {guest_dataset['email'][i]}"
    ]),
        metadata={"name": guest_dataset['name'][i]}
    )
     for i in range(len(guest_dataset))
]

/home/william/repos/agentic-rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
docs[0]

Document(id_='5d6887fe-58af-45ff-8e74-906d0510c7c7', embedding=None, metadata={'name': 'Ada Lovelace'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="Name: Ada Lovelace\nRelation: best friend\nDescription: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.\nEmail: ada.lovelace@example.com", path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}')

## Building the retriever

In [6]:
from llama_index.core.tools import FunctionTool
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever.from_defaults(nodes=docs)

def get_guest_info_retriever(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.retrieve(query)
    if results:
        return "\n\n".join([doc.text for doc in results[:3]])
    else:
        return "No matching guest information found."

# Initialize the tool
guest_info_tool = FunctionTool.from_defaults(get_guest_info_retriever)

## Building the Agent

In [7]:
!pip install llama-index-llms-huggingface-api


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [8]:
!pip install llama-index-llms-ollama


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [9]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.ollama import Ollama

# Initialize the Hugging Face model
llm = Ollama(
        model="llama3.2",
        temperature=0.1,
        request_timeout=120.0
    )

alfred = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool],
    llm=llm
)

In [10]:
# Example query Alfred might receive during the gala
response = await alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response)

🎩 Alfred's Response:
Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.

I do not have a relation with Marie Curie.


## Giving Alfred access to the web

In [11]:
!pip install llama-index-tools-duckduckgo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 2.4 MB/s eta 0:00:00a 0:00:01m

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [19]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.tools import FunctionTool

tool_spec = DuckDuckGoSearchToolSpec()
search_tool = FunctionTool.from_defaults(tool_spec.duckduckgo_full_search)

response = search_tool("Who's the current President of France?")
print(f"🔍 Search Tool Response: {response.raw_output[-1]['body']}")


🔍 Search Tool Response: Emmanuel Macron, born on December 21, 1977, is the current president of the French Republic.In office since May 14, 2017, he was re-elected on April 24, 2022 with 58.55% of the votes against his competitor Marine Le Pen. With a background in higher public service and investment banking, Macron approaches his mandate with a desire for change and innovation in various fields, including economics ...


## Custom weather search tool

In [20]:
import random
from llama_index.core.tools import FunctionTool

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool
weather_info_tool = FunctionTool.from_defaults(get_weather_info)

## AI Builders

In [ ]:
from llama_index.core.tools import FunctionTool
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"

hub_stats_tool = FunctionTool.from_defaults(get_hub_stats)

print(hub_stats_tool("facebook")) # Example: Get the most downloaded model by Facebook

The most downloaded model by facebook is facebook/esmfold_v1 with 21,289,130 downloads.


## Integrating Tools to Alfred

In [22]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.ollama import Ollama

llm = Ollama(
        model="llama3.2",
        temperature=0.1,
        request_timeout=120.0
    )
alfred = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool, search_tool, weather_info_tool, hub_stats_tool],
    llm=llm
)

In [24]:
response = await alfred.run("What's the weather like in Paris?")
print(f"🌤️ Alfred's Weather Response: {response}")

🌤️ Alfred's Weather Response: The current weather in Paris is mostly sunny with a gentle breeze. The temperature is around 20°C (68°F), making it a pleasant day to explore the city's famous landmarks and attractions. However, please note that this information may not be up-to-date or accurate, as my knowledge cutoff is December 2023. For the most current weather forecast, I recommend checking a reliable weather website or app.


In [25]:
response = await alfred.run("What is Facebook and what's their most popular model?")

print("🎩 Alfred's Response:")
print(response)

🎩 Alfred's Response:
Facebook is a social media platform where users can create profiles, connect with friends and family, share updates and photos, and join groups based on common interests.

As for their most popular model, the Facebook ESMFold-v1 model has been downloaded over 21 million times, making it one of the most widely used models on the platform. This model is specifically designed to predict protein folding structures, which is a critical task in understanding the function and behavior of proteins in living organisms.

The ESMFold-v1 model uses a combination of machine learning algorithms and physics-based methods to predict protein folding structures with high accuracy. It has been widely adopted by researchers and scientists in the field of structural biology and has contributed significantly to our understanding of protein function and disease mechanisms.

Overall, Facebook's ESMFold-v1 model is an important tool for researchers and scientists working in the field of st